In [11]:
import cv2 
import numpy as np 
import tensorflow as tf 
import json 

In [19]:
img=cv2.imread('/home/taibui/OpenCV/code/phieu.jpg',0)
#img=cv2.imshow('abc',img)
#cv2.waitKey(0)

height, width =img.shape[:2]
img = cv2.resize(img, (width/5, height/5), interpolation = cv2.INTER_AREA)
img.shape

(652, 489)

In [22]:
# threshold blur , lam mờ ảnh, với ngưỡng 
blur = cv2.GaussianBlur(img,(5,5),0)
thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)

In [20]:
#thresh=cv2.imshow('abc',thresh)
#cv2.waitKey(0)

255

In [5]:
#kernel = np.ones((3,3),np.uint8)
#erosion = cv2.erode(thresh,kernel,iterations = 1)
#cv2.imshow('asd',erosion)
#cv2.waitKey()

In [23]:
horizal=thresh
vertical=thresh 

scale_height=16  # xac dinh so dong doc
scale_long=12

height=int(img.shape[0]/scale_height)
large = int(img.shape[1]/scale_long)

print height,large


horizalStructure=cv2.getStructuringElement(cv2.MORPH_RECT,(large,1))
horizal=cv2.erode(horizal, horizalStructure,(-1,-1))
horizal=cv2.dilate(horizal, horizalStructure,(-1,-1))

verticalStructure=cv2.getStructuringElement(cv2.MORPH_RECT,(1,height))
vertical=cv2.erode(vertical,verticalStructure,(-1,-1))
vertical=cv2.dilate(vertical,verticalStructure,(-1,-1))

mask=vertical+horizal

cv2.imshow('abc',mask)
cv2.waitKey()

40 40


255

In [24]:
# tim vi tricua 1 obj trong anh
#contours : lưu trữ các đường biên tìm được
#hierarchy :  chứa thông tin về hình ảnh như số đường viền
#CV_RETR_EXTERNAL : cờ nó chỉ lấy ra những đường biên bên ngoài
# CV_CHAIN_APPROX_SIMPLE : nó sẽ nén đường viền trước khi lưu trữ
_,contours, hierachy= cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

max=-1
# tim khung hinh bao quanh max (khung dap an)  
for contour in contours:
    x,y,w,h= cv2.boundingRect(contour)
    if cv2.contourArea(contour) >max :
        x_max,y_max,w_max,h_max=x,y,w,h
        max=cv2.contourArea(contour)
        

In [25]:
table= img[y_max:y_max+h_max,x_max:x_max+w_max]

cv2.imshow('acv',table)
cv2.waitKey()


255

In [26]:
cropped_thresh_img=[]
cropped_origin_img=[]
contours_img=[]

num_row=10 # chia chieu doc thanh 10 dong, chia chieu ngang thanh 6 phan 
for i in range(1,num_row):
    thresh1=thresh[y_max+int(i*h_max/num_row):y_max+int((i+1)*h_max/num_row),x_max+int(w_max/6):x_max+int(w_max/2)]
    thresh1=cv2.resize(thresh1,(38,155), interpolation = cv2.INTER_AREA)
    _,contours_thresh1,hierarchy_thresh1=cv2.findContours(thresh1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    origin1=img[y_max+int(i*h_max/num_row):y_max+int((i+1)*h_max/num_row),x_max+int(w_max/6):x_max+int(w_max/2)]
    
    cropped_thresh_img.append(thresh1)
    cropped_origin_img.append(origin1)
    contours_img.append(contours_thresh1)

for i in range(1, num_row):
    thresh1 = thresh[y_max + int(i*h_max/num_row):y_max + int((i+1)*h_max/num_row), x_max + int(2*w_max/3):x_max +int(w_max)]
    _, contours_thresh1, hierarchy_thresh1 = cv2.findContours(thresh1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    origin1 = img[y_max + int(i*h_max/num_row):y_max + int((i+1)*h_max/num_row), x_max + int(2*w_max/3):x_max +int(w_max)]

    cropped_thresh_img.append(thresh1)
    cropped_origin_img.append(origin1)
    contours_img.append(contours_thresh1)
    
    print(thresh1.shape) 



(38, 155)
(37, 155)
(38, 155)
(37, 155)
(38, 155)
(37, 155)
(38, 155)
(37, 155)
(38, 155)


In [27]:
for i, contour_img in enumerate(contours_img):
    for cnt in contour_img:
        if cv2.contourArea(cnt) > 30:
            x,y,w,h = cv2.boundingRect(cnt)
            if x > cropped_origin_img[i].shape[1]*0.1 and x < cropped_origin_img[i].shape[1]*0.9:
                answer = cropped_origin_img[i][y:y+h, x:x+w]
                answer = cv2.threshold(answer, 160, 255, cv2.THRESH_BINARY_INV)[1]



In [29]:

cv2.imshow('asd',cropped_origin_img[12])
cv2.waitKey()

255

In [26]:
!pip3 install tflearn 

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [27]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected

network = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1])

network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = fully_connected(network, 1024, activation='relu')
network = dropout(network, 0.8)

network = fully_connected(network, N_CLASSES, activation='softmax')
network = regression(network)

model = tflearn.DNN(network)

ImportError: No module named tflearn